In [4]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
from scipy.stats import skewnorm
from ll_xy import lonlat_to_xy
from scipy.stats import linregress
import string
import cartopy
import cartopy.crs as ccrs
import pandas as pd
pd.set_option("mode.chained_assignment", None)
import tools

dep_l = pickle.load(open('line_depths_dict.p','rb'))

In [6]:
transect_count = 0

win_list = []
sum_list = []

for key in dep_l:
    
    station = dep_l[key]
    
    for date in station:

        transect_count += 1

        series = np.array(list(station[date]))

        n_series = (series - np.nanmean(series))

        std = np.nanstd(n_series)

        ns_series = n_series/std
        
        
        if date.month in [10,11,12,1,2,3]:
            win_list.append({'std':std,
                         'mean':np.nanmean(series),
                         'norm':ns_series,
                        })
        else:
            sum_list.append({'std':std,
                         'mean':np.nanmean(series),
                         'norm':ns_series,
                            })

win_df = pd.DataFrame(win_list)
sum_df = pd.DataFrame(sum_list)
all_df = pd.DataFrame(win_list + sum_list)

all_statistics = tools.get_statistics(all_df)

In [ ]:
bw = 0.1

bin_edges = np.arange(-2.5,3,bw)

bin_centres = bin_edges[:-1] + (bin_edges[1] - bin_edges[0])/2

flat = np.concatenate(df['norm']).ravel()

flat = flat[~np.isnan(flat)]

p, x = np.histogram(flat,
                    bins=bin_edges,
                    density=True,
                   )

fit = skewnorm.pdf(bin_centres,all_statistics['a'],all_statistics['loc'],all_statistics['scale'])

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(9,3))

ax1.scatter(df['mean'],df['std'],marker='x',color='crimson',alpha=0.5)

ax1.set_xlabel('Mean Snow Depth (cm)', fontsize='x-large')
ax1.set_ylabel('Standard Deviation \n Snow Depth (cm)', fontsize='x-large')

ax1.annotate(text='(a)', fontsize='xx-large', xy=(-0.1,1.05), xycoords= 'axes fraction')

linear_prediction = np.array([0,np.max(df['mean'])])*stats[0][0]

ax1.plot([0,np.max(df['mean'])], linear_prediction, color='k',lw=3, ls='--')

ax2.bar(bin_centres, p, color='crimson',width=0.1)

ax2.plot(bin_centres, fit,color='k',lw=3)

ax2.annotate(f"n = {all_statistics['n']}", xy=(0.95,0.95),xycoords='axes fraction',ha='right',va='top',fontsize='x-large')

ax2.set_ylabel('Probability Density', fontsize='x-large')
ax2.set_xlabel('Standard Deviations Away \n from Transect Mean Snow Depth', fontsize='x-large')
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")


ax2.annotate(text='(b)', fontsize='xx-large', xy=(-0.1,1.05), xycoords= 'axes fraction')

plt.savefig('/home/robbie/Dropbox/Apps/Overleaf/sub-km-snow-depth-dist/fig1.png', bbox_inches='tight',dpi=500)

plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(9,3))


fits = {}
statistics = {}

for df, season in zip([all_df, win_df, sum_df], ['all','winter','summer']):
    
    statistics[season] = get_statistics(df)

    bw = 0.1

    bin_edges = np.arange(-2.5,3,bw)

    bin_centres = bin_edges[:-1] + (bin_edges[1] - bin_edges[0])/2

    flat = np.concatenate(df['norm']).ravel()

    flat = flat[~np.isnan(flat)]

    p, x = np.histogram(flat,
                        bins=bin_edges,
                        density=True,
                       )

    fit = skewnorm.pdf(bin_centres,
                       statistics[season]['a'],
                       statistics[season]['loc'],
                       statistics[season]['scale'])
    
    fits[season] = {'fit':fit,
                    'flat':flat,
                    'bin_centres':bin_centres,
                    'p':p}
    
    if season in ['winter','summer']:
        
        color = 'dodgerblue' if season == 'winter' else 'darkorange'
        
        ax1.scatter(df['mean'],df['std'],color='lightgray', marker='o', alpha=0.5)
        
        linear_prediction = np.array([0,np.max(df['mean'])])*statistics[season]['stats']

        ax1.plot([0,np.max(all_df['mean'])], linear_prediction, color=color,lw=3, ls='--')
        
        ax2.bar(fits[season]['bin_centres'], fits[season]['p'],color=color,alpha=0.5,width=0.1,label=season)
        
        print(np.sum(np.multiply(0.1, fits[season]['p'])))
        
    ax2.plot(fits[season]['bin_centres'], fits[season]['fit'],color='k',lw=1)
    
ax2.legend(fontsize='large')
    
ax1.annotate(f"n = {statistics['summer']['n']}", xy = (0.05, 0.95),
             xycoords='axes fraction', va = 'top', fontsize='large',color='darkorange')
ax1.annotate(f"n = {statistics['winter']['n']}", xy = (0.05, 0.85),
             xycoords='axes fraction', va = 'top', fontsize='large',color='dodgerblue')

In [ ]:
pickle.dump(statistics, open('statistics.p','wb'))

In [ ]:
def get_std_from_mean(mean_depth):
    std = mean_depth * stats[0][0]
    return(std)

In [ ]:
mean_depth = 0.5
bin_width = 0.01

dep_bin_edges = np.arange(0,2,bin_width)

def depth_distribution_from_depth(mean_depth,dep_bin_edges,plot=False):

    dep_bin_centres = dep_bin_edges[:-1] + (dep_bin_edges[1] - dep_bin_edges[0])/2
    
    std = get_std_from_mean(mean_depth)
    
    std_bin_edges = (dep_bin_edges-mean_depth)/std
    std_bin_centres = std_bin_edges[:-1] + (std_bin_edges[1] - std_bin_edges[0])/2
    std_bw = np.nanmean(np.diff(std_bin_edges))
    
    fit = skewnorm.pdf(std_bin_centres,ae,loce,scalee)*std_bw
    
    return(dep_bin_centres, fit)
    
bc, ft = depth_distribution_from_depth(mean_depth,dep_bin_edges,plot=True)


plt.bar(bc,ft,width=bin_width,color='crimson')
plt.plot(bc, ft,lw=2,color='darkblue')
plt.ylabel('Proability Density',fontsize='x-large')
plt.xlabel('Snow Depth (m)',fontsize='x-large')
plt.axvline(mean_depth)
plt.show()